#### Imports/setup

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import ee
import os
import pandas as pd
import geopandas as gpd
# import json
import geemap
import time
import functools
import os
import sys
# from datetime import datetime

import requests # may not be needed
from sidecar import Sidecar

from modules.gee_initialize import initialize_ee 
initialize_ee()

from modules.file_to_ceo import (
get_ceo_url, 
whisp_stats_shapefile_and_ceo_url
)

from parameters.config_runtime import (
    geo_id_column, 
    out_file_wide, 
    out_shapefile
)
from modules.agstack_to_gee import (
    start_agstack_session,
    get_agstack_token,
    geo_id_or_ids_to_feature_collection,
    shapefile_to_ee_feature_collection,
    register_fc_and_set_geo_id,
    register_fc_and_append_to_csv,
    add_geo_ids_to_feature_col_from_lookup_df,
    add_geo_ids_to_feature_col_from_lookup_csv
)

from modules.stats import get_stats,reformat_whisp_fc,get_stats_formatted

from parameters.config_asr_url_info import asset_registry_base, user_registry_base

from parameters.config_asr_credentials import email, password

from modules.utils import (
    collection_properties_to_df,
    remove_geometry_from_feature_collection,
    get_centroid#,
    # get_scale_from_image
)

def whisp_stats_as_df (roi): 
    %store roi
    %run process_whisp_stats.ipynb   
    return df_out
   

Earth Engine has been initialized with the specified credentials.


##### Set up asset registry

In [4]:
session = start_agstack_session(email,password,user_registry_base)

token = get_agstack_token(email, password, asset_registry_base)

#### Inputs : with GEOIDS

##### Input: list of Geo ids

In [5]:
example_ids = ['88bec54ad04804f5b1fafbc131266640a129be2840fa6797cda358d7e831b907']

GHA_ids = ['88bec54ad04804f5b1fafbc131266640a129be2840fa6797cda358d7e831b907', 
        '97408ef7bdac487e4a42e4abf20492b786310889fd4b0478603e2d0004c40bfb']

IDN_ids = ['c288d6c94efa9011c0e3452af9f7fa0941661377030e10d29c68764617f9816d', 
       '8e2accea7ddbb84b7f6001e00bcb60f57f563c80633b53859993522a6f05727a']

CIV_ids = ['0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f665ea83ac9aad3512918',
            'fa2aff0d60cf1bc0e1f1dd4b91daf932940c31c021ca1b84f5b9445855eef02f']

all_geo_ids= GHA_ids + IDN_ids + CIV_ids
#all_geo_ids= example_ids

In [6]:
#make into a feature collection
roi_geoid = geo_id_or_ids_to_feature_collection(
    all_geo_ids=all_geo_ids, 
    geo_id_column=geo_id_column, 
    session=session)

#### Inputs : without GEOIDS

##### Input: GEE asset (feature collection) 

In [7]:
example_plot_feature_col_asset = 'projects/ee-andyarnellgee/assets/p0004_commodity_mapper_support/seg_civ_PCBM'

# Define the fraction of features you want to sample (e.g., 0.5 for 50%)
sampleFraction = 0.0010

In [8]:
fc = ee.FeatureCollection(example_plot_feature_col_asset)

randomColumn = 'random'
fc_r = fc.randomColumn(randomColumn,4)

# Filter the feature collection based on the random column to get the sample
fc_sample = fc_r.filter(ee.Filter.lt(randomColumn, sampleFraction));

fc = fc_sample



##### if registering many polygons 

In [9]:
#step 1: create a csv lookup table with geo ids 
register_fc_and_append_to_csv(
    feature_col=fc,
    geo_id_column="Geo_id",
    output_lookup_csv="temp_geo_id_lookup.csv",
    join_id_column="system:index",
    token=token,
    session=session,
    asset_registry_base=asset_registry_base,
    override_checks=False,
    remove_temp_csv=False,
    debug=True)


Check passed: feature collection and table same size: 7 rows
Number without geo ids:0. 
 Processing started...
Backup file copied successfully to: backup_csvs/temp_geo_id_lookup_20240423_083313.csv
Done


In [10]:
#Step 2: join csv lookup (with geo ids) to feature collection
join_id_column = "system:index"

roi_asset = add_geo_ids_to_feature_col_from_lookup_csv(
    fc,
    "temp_geo_id_lookup.csv",
    join_id_column,
    geo_id_column,
    override_checks=False,
    remove_other_properties=True,
    debug=True)


Check passed: feature collection and table same size: 7 rows
dropping rows without values (NaN etc) from input table. Before: 7 After: 7 
 processing...
Finished. 
 New count of Geo_id values in feature collection: 7 (from total of 7 features)


##### if registering a few polygons

In [11]:
# # add geo ids directly to feature collection 
# roi_asset = register_fc_and_set_geo_id(
#     fc,
#     geo_id_column,
#     token,
#     session,
#     asset_registry_base,
#     debug=True
# )

# print ("done")

In [11]:
roi_asset

##### Input: shapefile

In [12]:
# example_shapefile = /path_to/my_shapefile
example_shapefile = out_shapefile

In [15]:
# convert shapefile to feature collection
roi_shapefile = shapefile_to_ee_feature_collection(example_shapefile) 

# remove Geo_id (for demo purpopses only)
roi_shapefile = roi_shapefile.map(lambda feature: feature.set("Geo_id", None)) 


#### Get plot boundaries

In [11]:
# roi = roi_geoid
# roi = roi_asset
roi = roi_shapefile
#roi = roi_polygon
#roi = roi_points

roi # to check it looks ok

In [12]:
register_fc_and_append_to_csv(
    feature_col=roi,
    geo_id_column="Geo_id",
    output_lookup_csv="temp_geo_id_lookup.csv",
    join_id_column="system:index",
    token=token,
    session=session,
    asset_registry_base=asset_registry_base,
    override_checks=False,
    remove_temp_csv=False,
    debug=True)

Number without geo ids:6. 
 Processing started...
Matched existing field (failed to register). Status code: 400 Using pre-existing geo id:  88bec54ad04804f5b1fafbc131266640a129be2840fa6797cda358d7e831b907
0
Matched existing field (failed to register). Status code: 400 Using pre-existing geo id:  97408ef7bdac487e4a42e4abf20492b786310889fd4b0478603e2d0004c40bfb
1
Matched existing field (failed to register). Status code: 400 Using pre-existing geo id:  c288d6c94efa9011c0e3452af9f7fa0941661377030e10d29c68764617f9816d
2
Matched existing field (failed to register). Status code: 400 Using pre-existing geo id:  8e2accea7ddbb84b7f6001e00bcb60f57f563c80633b53859993522a6f05727a
3
Matched existing field (failed to register). Status code: 400 Using pre-existing geo id:  0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f665ea83ac9aad3512918
4
Matched existing field (failed to register). Status code: 400 Using pre-existing geo id:  fa2aff0d60cf1bc0e1f1dd4b91daf932940c31c021ca1b84f5b9445855eef02f
5
Backup f

### Whisp it

In [20]:
df = whisp_stats_as_df(roi)

Stored 'roi' (FeatureCollection)
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Datasets present in lookup:  True
['Geo_id', 'Area_ha', 'Country']
geo id present
...stats processing started
...ee to df processing time: 4.738058090209961 seconds


### Display it

In [21]:
df

,Geo_id,Area_ha,Country,GLAD_Primary,TMF_undist,JAXA_FNF_2020,GFC_TC_2020,ESA_TC_2020,TMF_disturbed,TMF_plant,...,TMF_distur,RADD_alert,EUFO_2020,PLOTID,Oil_palm_F,Oil_palm_D,GLAD_Prima,GFC_TC_202,ESA_TC_202,GLAD_LULC_
0,88bec54ad04804f5b1fafbc131266640a129be2840fa67...,1.9,GHA,0.000000,0.000000,1.938262,0.386413,1.938262,0.358137,0.000000,...,18,-,NaN,1,0,0,0,20,100,85
1,97408ef7bdac487e4a42e4abf20492b786310889fd4b04...,16.6,GHA,14.493296,15.791776,16.602398,16.602398,16.602398,0.810621,0.000000,...,5,True,NaN,2,0,0,87,100,100,89
2,c288d6c94efa9011c0e3452af9f7fa0941661377030e10...,31.2,IDN,0.000000,0.000000,1.717910,0.175786,29.087104,0.000000,31.210226,...,0,-,NaN,3,82,99,0,1,93,71
3,8e2accea7ddbb84b7f6001e00bcb60f57f563c80633b53...,20.9,IDN,0.000000,6.472015,20.098499,11.239226,20.854253,9.279256,0.000000,...,44,-,NaN,4,51,0,0,54,100,96
4,0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f66...,8.3,CIV,0.000504,0.022910,7.833959,5.084635,6.402215,0.329385,0.000000,...,4,-,NaN,5,0,0,0,61,77,21
5,fa2aff0d60cf1bc0e1f1dd4b91daf932940c31c021ca1b...,3.6,CIV,0.000000,0.000000,3.613207,1.658091,3.544055,0.232675,0.000000,...,6,-,NaN,6,0,0,0,46,98,100


#### Export it to CSV file

In [22]:
#export output csv
df.to_csv(path_or_buf=out_file_wide,header=True)

print ("output csv: ", out_file_wide)


output csv:  /home/sepal-user/whisp/whisp_output_table.csv


#### Export it to CEO link

Temporary login details available for demo purposes here: modules/config_ceo.py

NB first 10 characters of each column names need to be distinct or you will get an error - as the names arew truncated to this length as per the (current) requirements of the import process for CEO.

In [24]:
ceo_url = whisp_stats_shapefile_and_ceo_url(roi, df, plot_id_column, out_shapefile)

print('CEO URL:', ceo_url) 

ValueError: GeoDataFrame cannot contain duplicated column names.